In [7]:
import pandas as pd
import numpy as np
import os
import glob
pd.options.mode.chained_assignment = None  # default='warn'


In [8]:
base_dir = r'/Users/sweetpotato/Desktop/Data Visualisation & Analytics/food-supply-reporting/data_in/food_daily_consumption'


In [9]:
# directory of csv files
data_in = glob.glob(os.path.join(base_dir, '*.csv'))

# selected columns from data input
columns = ['Area Code (FAO)', 'Area', 'Element Code',
           'Element', 'Item Code', 'Item', 'Year', 'Unit', 'Value']
renamed_columns = ['Area_Code', 'Area', 'Element_Code',
                   'Element', 'Item_Code', 'Item', 'Year', 'Unit', 'Value']

# read and union all the data inputs
df = pd.DataFrame()
for file in data_in:
    data = pd.read_csv(file)
    data = data[columns]
    df = pd.concat([df, data], ignore_index=True)

# rename df
df.columns = renamed_columns


In [10]:
df.head()


,Area_Code,Area,Element_Code,Element,Item_Code,Item,Year,Unit,Value
0,2,Afghanistan,664,Food supply (kcal/capita/day),2511,Wheat and products,2000,kcal/capita/day,1079.0
1,2,Afghanistan,664,Food supply (kcal/capita/day),2511,Wheat and products,2001,kcal/capita/day,1068.0
2,2,Afghanistan,664,Food supply (kcal/capita/day),2511,Wheat and products,2002,kcal/capita/day,1069.0
3,2,Afghanistan,664,Food supply (kcal/capita/day),2511,Wheat and products,2003,kcal/capita/day,1144.0
4,2,Afghanistan,664,Food supply (kcal/capita/day),2511,Wheat and products,2004,kcal/capita/day,1156.0


In [11]:
df[['Element', 'Element_Code', 'Unit']].drop_duplicates().reset_index(drop=True)


,Element,Element_Code,Unit
0,Food supply (kcal/capita/day),664,kcal/capita/day
1,Protein supply quantity (g/capita/day),674,g/capita/day
2,Fat supply quantity (g/capita/day),684,g/capita/day


In [12]:
# Dictionary of each element and related information
element_dict = {
    'food_energy': {'fao_ele_code': 664, 'element_code': 100,
                    'element_name': 'Food Supply Energy', 'unit': 'kcal/capita/day'},
    'food_quant': {'element_code': 110,
                   'element_name': 'Food Supply Quantity', 'unit': 'g/capita/day'},
    'protein_energy': {'element_code': 200,
                       'element_name': 'Protein Supply Energy', 'unit': 'kcal/capita/day'},
    'protein_quant': {'fao_ele_code': 674, 'element_code': 210,
                      'element_name': 'Protein Supply Quantity', 'unit': 'g/capita/day'},
    'fat_energy': {'element_code': 300,
                   'element_name': 'Fat Supply Energy', 'unit': 'kcal/capita/day'},
    'fat_quant': {'fao_ele_code': 684, 'element_code': 310,
                  'element_name': 'Fat Supply Quantity', 'unit': 'g/capita/day'},
    'carbs_energy': {'element_code': 400,
                     'element_name': 'Carbs Supply Energy', 'unit': 'kcal/capita/day'},
    'carbs_quant': {'element_code': 410,
                    'element_name': 'Carbs Supply Quantity', 'unit': 'g/capita/day'}
}


In [13]:
# Function that takes data input and returns data output which complies to the naming convention defined in element_dict
def Revised(input, element_name):
    selected_dict = element_dict[element_name]
    output = input
    output['Element'] = selected_dict['element_name']
    output['Element_Code'] = selected_dict['element_code']
    output['Unit'] = selected_dict['unit']
    return output


In [14]:
# Function that extracts the existing data provided by FAO Stats (food energy, protein quant and fat quant) and returns the clean data output complied to element_dict
# Posible element_name variables of this function include food_energy, pretoin_quant and fat_quant
def RevisedRaw(element_name):
    selected_dict = element_dict[element_name]
    fao_ele_code = selected_dict['fao_ele_code']
    input = df[df['Element_Code'] == fao_ele_code].reset_index(drop=True)
    output = Revised(input, element_name)
    return output


In [15]:
# Convert protein and fat energy into protein and fat quant based on defined formula
def EnergyFromQuant(element_name):
    selected_dict = element_dict[element_name]
    if element_name == 'protein_energy':
        input = RevisedRaw('protein_quant')
        input['Value'] = input['Value']*4
    elif element_name == 'fat_energy':
        input = RevisedRaw('fat_quant')
        input['Value'] = input['Value']*9
    output = input
    output['Element'] = selected_dict['element_name']
    output['Element_Code'] = selected_dict['element_code']
    output['Unit'] = selected_dict['unit']
    return output


In [16]:
# Return clean data frame for each element input
def ElementDataFrame(element_name):
    # convert the data from the raw fao data source
    if element_name in ('food_energy', 'protein_quant', 'fat_quant'):
        return RevisedRaw(element_name)
    elif element_name in ('protein_energy', 'fat_energy'):
        return EnergyFromQuant(element_name)


In [17]:
# From the raw data provided by FAO Stats and further calculations, we can create the following data frames
food_energy = ElementDataFrame('food_energy')
protein_energy = ElementDataFrame('protein_energy')
fat_energy = ElementDataFrame('fat_energy')
protein_quant = ElementDataFrame('protein_quant')
fat_quant = ElementDataFrame('fat_quant')


Calculate Carbs energy and Carbs qunatity

In [18]:
# Next is to compute the missing dataframes which include food_quant, carbs_energy and carbs_quant
energy = pd.concat([food_energy, protein_energy,
                   fat_energy], ignore_index=True)
energy_pivot = pd.pivot_table(energy, index=['Area_Code', 'Area', 'Item_Code', 'Item',
                              'Year', 'Unit'], columns='Element', values='Value', aggfunc=np.sum).reset_index()


In [19]:
energy_pivot.head()


Element,Area_Code,Area,Item_Code,Item,Year,Unit,Fat Supply Energy,Food Supply Energy,Protein Supply Energy
0,1,Armenia,2511,Wheat and products,2000,kcal/capita/day,34.74,1218.0,145.68
1,1,Armenia,2511,Wheat and products,2001,kcal/capita/day,33.39,1173.0,140.32
2,1,Armenia,2511,Wheat and products,2002,kcal/capita/day,31.32,1095.0,131.04
3,1,Armenia,2511,Wheat and products,2003,kcal/capita/day,33.03,1134.0,135.60
4,1,Armenia,2511,Wheat and products,2004,kcal/capita/day,37.08,1260.0,151.16


In [20]:
# Calculate the missing fields based on defined formulas
energy_pivot['Carbs Supply Energy'] = energy_pivot['Food Supply Energy'] - \
    (energy_pivot['Protein Supply Energy']+energy_pivot['Fat Supply Energy'])
energy_pivot['Carbs Supply Quantity'] = energy_pivot['Carbs Supply Energy']/4
energy_pivot['Food Supply Quantity'] = energy_pivot['Protein Supply Energy'] / \
    4+energy_pivot['Fat Supply Energy']/9+energy_pivot['Carbs Supply Energy']/4


In [21]:
# Create carbs_energy df
carbs_energy = energy_pivot[energy_pivot.columns.difference(
    ['Fat Supply Energy', 'Food Supply Energy', 'Protein Supply Energy', 'Carbs Supply Quantity', 'Food Supply Quantity'])]
carbs_energy = Revised(carbs_energy, 'carbs_energy')
carbs_energy.rename(columns={'Carbs Supply Energy': 'Value'}, inplace=True)


In [22]:
# Create carbs_quant df
carbs_quant = energy_pivot[energy_pivot.columns.difference(
    ['Fat Supply Energy', 'Food Supply Energy', 'Protein Supply Energy', 'Carbs Supply Energy', 'Food Supply Quantity'])]
carbs_quant = Revised(carbs_quant, 'carbs_quant')
carbs_quant.rename(columns={'Carbs Supply Quantity': 'Value'}, inplace=True)


In [23]:
# Create food_quant df
food_quant = energy_pivot[energy_pivot.columns.difference(
    ['Fat Supply Energy', 'Food Supply Energy', 'Protein Supply Energy', 'Carbs Supply Energy', 'Carbs Supply Quantity'])]
food_quant = Revised(food_quant, 'food_quant')
food_quant.rename(columns={'Food Supply Quantity': 'Value'}, inplace=True)


In [24]:
energy = pd.concat([food_energy, protein_energy, fat_energy,
                   carbs_energy], ignore_index=True)
energy


,Area_Code,Area,Element_Code,Element,Item_Code,Item,Year,Unit,Value
0,2,Afghanistan,100,Food Supply Energy,2511,Wheat and products,2000,kcal/capita/day,1079.00
1,2,Afghanistan,100,Food Supply Energy,2511,Wheat and products,2001,kcal/capita/day,1068.00
2,2,Afghanistan,100,Food Supply Energy,2511,Wheat and products,2002,kcal/capita/day,1069.00
3,2,Afghanistan,100,Food Supply Energy,2511,Wheat and products,2003,kcal/capita/day,1144.00
4,2,Afghanistan,100,Food Supply Energy,2511,Wheat and products,2004,kcal/capita/day,1156.00
...,...,...,...,...,...,...,...,...,...
1025905,351,China,400,Carbs Supply Energy,2899,Miscellaneous,2014,kcal/capita/day,-0.04
1025906,351,China,400,Carbs Supply Energy,2899,Miscellaneous,2015,kcal/capita/day,-0.04
1025907,351,China,400,Carbs Supply Energy,2899,Miscellaneous,2016,kcal/capita/day,0.00
1025908,351,China,400,Carbs Supply Energy,2899,Miscellaneous,2017,kcal/capita/day,0.00


In [25]:
quantity = pd.concat(
    [food_quant, protein_quant, fat_quant, carbs_quant], ignore_index=True)
quantity


,Area,Area_Code,Value,Item,Item_Code,Unit,Year,Element,Element_Code
0,Armenia,1,299.6750,Wheat and products,2511,g/capita/day,2000,Food Supply Quantity,110
1,Armenia,1,288.6125,Wheat and products,2511,g/capita/day,2001,Food Supply Quantity,110
2,Armenia,1,269.4000,Wheat and products,2511,g/capita/day,2002,Food Supply Quantity,110
3,Armenia,1,278.9125,Wheat and products,2511,g/capita/day,2003,Food Supply Quantity,110
4,Armenia,1,309.8500,Wheat and products,2511,g/capita/day,2004,Food Supply Quantity,110
...,...,...,...,...,...,...,...,...,...
1026032,China,351,-0.0100,Miscellaneous,2899,g/capita/day,2014,Carbs Supply Quantity,410
1026033,China,351,-0.0100,Miscellaneous,2899,g/capita/day,2015,Carbs Supply Quantity,410
1026034,China,351,0.0000,Miscellaneous,2899,g/capita/day,2016,Carbs Supply Quantity,410
1026035,China,351,0.0000,Miscellaneous,2899,g/capita/day,2017,Carbs Supply Quantity,410


In [26]:
df = pd.concat([energy, quantity], ignore_index=True)


In [27]:
# # 1. Setting up
# from google.cloud import bigquery

# # 2. Downloading the data
# filename =  '/Users/sweetpotato/Desktop/Data Visualisation & Analytics/code/may-eleventh-e207a3fd73f9.json' #BQ access key
# bq_client = bigquery.Client.from_service_account_json(filename)
# table_config = bigquery.job.LoadJobConfig(write_disposition='WRITE_TRUNCATE')
# table_config.autodetect = True


# # 3. Pushing the data back to BQ
# job = bq_client.load_table_from_dataframe(
#         df,'may-eleventh.food_supply_reporting.fao_food_supply',job_config=table_config)
# job.result()  # Waits for table load to complete.
